# Deep Belief Model

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix

# Load dataset
print("📂 Loading dataset...")
df = pd.read_csv("df.csv")  
print(f"✅ Dataset Loaded! Shape: {df.shape}")

# Encode categorical labels
print("\n🔄 Encoding categorical labels...")
label_encoder = LabelEncoder()
df["Class"] = label_encoder.fit_transform(df["Class"])
df["theft"] = label_encoder.fit_transform(df["theft"])
print("✅ Encoding complete!")

# Feature selection
feature_cols = [
    "Electricity:Facility [kW](Hourly)", "Fans:Electricity [kW](Hourly)", "Cooling:Electricity [kW](Hourly)",
    "Heating:Electricity [kW](Hourly)", "InteriorLights:Electricity [kW](Hourly)", "InteriorEquipment:Electricity [kW](Hourly)",
    "Gas:Facility [kW](Hourly)", "Heating:Gas [kW](Hourly)", "InteriorEquipment:Gas [kW](Hourly)",
    "Water Heater:WaterSystems:Gas [kW](Hourly)"
]

X = df[feature_cols].values
y = df["theft"].values

# Check class distribution
print("\n📊 Class Distribution Before Train-Test Split:")
print(Counter(y))

# Split dataset
print("\n✂️ Splitting dataset...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print(f"✅ Split complete! Train size: {X_train.shape}, Test size: {X_test.shape}")

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define LightGBM model & parameter grid
lgb_params = {
    "n_estimators": [100, 200, 300],  
    "learning_rate": [0.01, 0.05, 0.1],  
    "max_depth": [10, 20, -1],  
    "num_leaves": [31, 50, 100],  
    "boosting_type": ["gbdt"],  
    "objective": ["multiclass"],  
    "num_class": [len(np.unique(y))],  
    "metric": ["multi_logloss"],  
}

# Perform GridSearchCV
print("\n🚀 Starting GridSearchCV for LightGBM...")
lgb_model = lgb.LGBMClassifier()
lgb_grid = GridSearchCV(lgb_model, lgb_params, cv=2, scoring="f1_weighted", n_jobs=-1)
lgb_grid.fit(X_train, y_train)
print("✅ LightGBM GridSearch completed!")
print(f"🎯 Best LightGBM Parameters: {lgb_grid.best_params_}")

# Train final model
best_lgb = lgb_grid.best_estimator_

# Predictions
print("\n📢 Making predictions...")
y_pred = best_lgb.predict(X_test)
y_pred_proba = best_lgb.predict_proba(X_test)

# Evaluation function
def evaluate_model(y_true, y_pred, y_proba, model_name):
    results = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "F1-score": f1_score(y_true, y_pred, average="weighted"),
        "Kappa": cohen_kappa_score(y_true, y_pred),
        "AUC": roc_auc_score(y_true, y_proba, multi_class="ovr")
    }
    print(f"\n🔍 **{model_name} Results:**")
    print(results)
    print(f"\n📊 Confusion Matrix ({model_name}):\n", confusion_matrix(y_true, y_pred))

# Evaluate model
print("\n📊 Evaluating LightGBM...")
evaluate_model(y_test, y_pred, y_pred_proba, "LightGBM")

print("\n✅ **Training & Evaluation Complete!** 🚀")


📂 Loading dataset...
✅ Dataset Loaded! Shape: (560655, 13)

🔄 Encoding categorical labels...
✅ Encoding complete!

📊 Class Distribution Before Train-Test Split:
Counter({0: 331824, 1: 51083, 3: 44349, 4: 41460, 6: 35413, 5: 33553, 2: 22973})

✂️ Splitting dataset...
✅ Split complete! Train size: (448524, 10), Test size: (112131, 10)

🚀 Starting GridSearchCV for LightGBM...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002398 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2544
[LightGBM] [Info] Number of data points in the train set: 448524, number of used features: 10
[LightGBM] [Info] Start training from score -0.524502
[LightGBM] [Info] Start training from score -2.395639
[LightGBM] [Info] Start training from score -3.194808
[LightGBM] [Info] Start training from score -2.537021
[LightGBM] [Info] Start training from score -2.604377


In [5]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.utils.class_weight import compute_class_weight

# Load dataset
print("📂 Loading dataset...")
df = pd.read_csv("df.csv")  
print(f"✅ Dataset Loaded! Shape: {df.shape}")

# Encode categorical labels
print("\n🔄 Encoding categorical labels...")
label_encoder = LabelEncoder()
df["Class"] = label_encoder.fit_transform(df["Class"])
df["theft"] = label_encoder.fit_transform(df["theft"])
print("✅ Encoding complete!")

# Feature selection
feature_cols = [
    "Electricity:Facility [kW](Hourly)", "Fans:Electricity [kW](Hourly)", "Cooling:Electricity [kW](Hourly)",
    "Heating:Electricity [kW](Hourly)", "InteriorLights:Electricity [kW](Hourly)", "InteriorEquipment:Electricity [kW](Hourly)",
    "Gas:Facility [kW](Hourly)", "Heating:Gas [kW](Hourly)", "InteriorEquipment:Gas [kW](Hourly)",
    "Water Heater:WaterSystems:Gas [kW](Hourly)"
]

X = df[feature_cols].values
y = df["theft"].values

# Check class distribution before training
print("\n📊 Class Distribution Before Train-Test Split:")
print(Counter(y))

# Split dataset
print("\n✂️ Splitting dataset...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print(f"✅ Split complete! Train size: {X_train.shape}, Test size: {X_test.shape}")

# Feature Selection: Select Best Features
print("\n🔍 Selecting Best Features...")
selector = SelectKBest(score_func=mutual_info_classif, k=8)  # Keep 8 best features
X_train = selector.fit_transform(X_train, y_train)
X_test = selector.transform(X_test)
print(f"✅ Feature Selection Complete! New Shape: {X_train.shape}")

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Compute Class Weights
print("\n⚖️ Computing Class Weights...")
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
print(f"✅ Class Weights: {class_weight_dict}")

# Define LightGBM Model with Class Weights & Regularization
lgb_model = lgb.LGBMClassifier(
    boosting_type="gbdt",
    objective="multiclass",
    num_class=len(np.unique(y)),
    n_estimators=300,  
    learning_rate=0.05,
    max_depth=20,  
    num_leaves=50,  
    min_data_in_leaf=50,  
    reg_lambda=0.5,  
    class_weight="balanced",  # ✅ Using class weights correctly
    random_state=42
)

# Train LightGBM with Early Stopping
print("\n🚀 Training LightGBM Model with Early Stopping...")
lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],  
    eval_metric="multi_logloss",
    callbacks=[
        lgb.early_stopping(30),  # Stop if no improvement in 30 rounds
        lgb.log_evaluation(100)  # Log every 100 iterations
    ]
)

print("✅ LightGBM Training Completed!")

# Predictions
print("\n📢 Making predictions...")
y_pred = lgb_model.predict(X_test)
y_pred_proba = lgb_model.predict_proba(X_test)

# Evaluation function
def evaluate_model(y_true, y_pred, y_proba, model_name):
    results = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "F1-score": f1_score(y_true, y_pred, average="weighted"),
        "Kappa": cohen_kappa_score(y_true, y_pred),
        "AUC": roc_auc_score(y_true, y_proba, multi_class="ovr")
    }
    print(f"\n🔍 **{model_name} Results:**")
    print(results)
    print(f"\n📊 Confusion Matrix ({model_name}):\n", confusion_matrix(y_true, y_pred))

# Evaluate model
print("\n📊 Evaluating LightGBM...")
evaluate_model(y_test, y_pred, y_pred_proba, "Optimized LightGBM")

print("\n✅ **Training & Evaluation Complete!** 🚀")


📂 Loading dataset...
✅ Dataset Loaded! Shape: (560655, 13)

🔄 Encoding categorical labels...
✅ Encoding complete!

📊 Class Distribution Before Train-Test Split:
Counter({0: 331824, 1: 51083, 3: 44349, 4: 41460, 6: 35413, 5: 33553, 2: 22973})

✂️ Splitting dataset...
✅ Split complete! Train size: (448524, 10), Test size: (112131, 10)

🔍 Selecting Best Features...
✅ Feature Selection Complete! New Shape: (448524, 8)

⚖️ Computing Class Weights...
✅ Class Weights: {0: 0.24137383604570628, 1: 1.5678874677088395, 2: 3.4864978312578705, 3: 1.805993887732381, 4: 1.931827579077941, 5: 2.3870229535766176, 6: 2.2617316322928747}

🚀 Training LightGBM Model with Early Stopping...
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testi

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import shap
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, roc_auc_score, confusion_matrix

# ✅ Load dataset
print("📂 Loading dataset...")
df = pd.read_csv("df.csv")  
print(f"✅ Dataset Loaded! Shape: {df.shape}")

# ✅ Encode categorical labels
print("\n🔄 Encoding categorical labels...")
label_encoder = LabelEncoder()
df["Class"] = label_encoder.fit_transform(df["Class"])
df["theft"] = label_encoder.fit_transform(df["theft"])
print("✅ Encoding complete!")

# ✅ Feature selection
feature_cols = [
    "Electricity:Facility [kW](Hourly)", "Fans:Electricity [kW](Hourly)", "Cooling:Electricity [kW](Hourly)",
    "Heating:Electricity [kW](Hourly)", "InteriorLights:Electricity [kW](Hourly)", "InteriorEquipment:Electricity [kW](Hourly)",
    "Gas:Facility [kW](Hourly)", "Heating:Gas [kW](Hourly)", "InteriorEquipment:Gas [kW](Hourly)",
    "Water Heater:WaterSystems:Gas [kW](Hourly)"
]

X = df[feature_cols].values
y = df["theft"].values

# ✅ Check class distribution before training
print("\n📊 Class Distribution Before Train-Test Split:")
print(Counter(y))

# ✅ Split dataset
print("\n✂️ Splitting dataset...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
print(f"✅ Split complete! Train size: {X_train.shape}, Test size: {X_test.shape}")

# ✅ Apply SMOTE (Synthetic Minority Over-sampling)
print("\n🆕 Applying Intelligent SMOTE...")
smote = SMOTE(sampling_strategy={1: 70000, 2: 60000, 3: 70000, 4: 70000, 5: 60000, 6: 60000}, random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)
print(f"✅ SMOTE applied! New Train size: {X_train.shape}")

# ✅ Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# ✅ Define LightGBM model & parameter grid
lgb_params = {
    "n_estimators": [150, 200],  
    "learning_rate": [0.01, 0.1],  
    "max_depth": [10, 20, -1],  
    "num_leaves": [50],  
    "min_data_in_leaf": [20],  
    "colsample_bytree": [0.7, 0.9],  
    "subsample": [0.7],  
    "boosting_type": ["gbdt"],  
    "objective": ["multiclass"],  
    "num_class": [len(np.unique(y))],  
    "metric": ["multi_logloss"],  
}

# ✅ Perform GridSearchCV
print("\n🚀 Starting GridSearchCV for LightGBM...")
lgb_model = lgb.LGBMClassifier()
lgb_grid = GridSearchCV(lgb_model, lgb_params, cv=3, scoring="f1_weighted", n_jobs=-1, verbose=1)
lgb_grid.fit(X_train, y_train)
print("✅ LightGBM GridSearch completed!")
print(f"🎯 Best LightGBM Parameters: {lgb_grid.best_params_}")

# ✅ Train final model with best parameters
best_lgb = lgb_grid.best_estimator_

# ✅ Optimized SHAP Calculation (Only Uses 1000 Samples)
print("\n🔍 Calculating Feature Importance using SHAP (Limited Samples)...")
X_sample = X_train[:1000]  # Use only 1000 samples

explainer = shap.Explainer(best_lgb, X_sample)
shap_values = explainer(X_sample)

shap.summary_plot(shap_values, X_sample, feature_names=feature_cols)  # SHAP Summary Plot

# ✅ Predictions
print("\n📢 Making predictions...")
y_pred = best_lgb.predict(X_test)
y_pred_proba = best_lgb.predict_proba(X_test)

# ✅ Evaluation function
def evaluate_model(y_true, y_pred, y_proba, model_name):
    results = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "F1-score": f1_score(y_true, y_pred, average="weighted"),
        "Kappa": cohen_kappa_score(y_true, y_pred),
        "AUC": roc_auc_score(y_true, y_proba, multi_class="ovr")
    }
    print(f"\n🔍 **{model_name} Results:**")
    print(results)
    print(f"\n📊 Confusion Matrix ({model_name}):\n", confusion_matrix(y_true, y_pred))

# ✅ Evaluate final model
print("\n📊 Evaluating Improved LightGBM...")
evaluate_model(y_test, y_pred, y_pred_proba, "Improved LightGBM")

print("\n✅ **Training & Evaluation Complete!** 🚀")


📂 Loading dataset...
✅ Dataset Loaded! Shape: (560655, 13)

🔄 Encoding categorical labels...
✅ Encoding complete!

📊 Class Distribution Before Train-Test Split:
Counter({0: 331824, 1: 51083, 3: 44349, 4: 41460, 6: 35413, 5: 33553, 2: 22973})

✂️ Splitting dataset...
✅ Split complete! Train size: (448524, 10), Test size: (112131, 10)

🆕 Applying Intelligent SMOTE...
✅ SMOTE applied! New Train size: (655459, 10)

🚀 Starting GridSearchCV for LightGBM...
Fitting 3 folds for each of 24 candidates, totalling 72 fits


KeyboardInterrupt: 